In [1]:
pip install install infa_repo_qry


The following command must be run outside of the IPython shell:

    $ pip install install infa_repo_qry

The Python package manager (pip) can only be used from outside of IPython.
Please reissue the `pip` command in a separate terminal or command prompt.

See the Python documentation for more information on how to install packages:

    https://docs.python.org/3/installing/


In [2]:
!pip install install infa_repo_qry

ERROR: Could not find a version that satisfies the requirement install (from versions: none)
ERROR: No matching distribution found for install


In [4]:
print('sreenu')

sreenu


In [ ]:
import pyodbc,sys,smtplib,os

##################################################
##Written by Bob Brook  4/18/2008
##This script is intended to be executed, only if the Informatica job fails
##If there is a non zero value returned to autosys, this script will extract
##from the Informatica repository the values associated with the failure.

##sample complete connection string
##insert="driver={SQL Server};server=sdwpce1q\edh;database=infapc;Trusted_Connection=yes"

##parametized
insert="driver={SQL Server};server=DB_SERVER;database=DB_INFA;Trusted_Connection=yes"
#cnxn = pyodbc.connect('driver={SQL Server};server=SDWPCE1D\PLM;database=EnterpriseControls;Trusted_Connection=yes')
#cnxn=pyodbc.connect('DSN=EnterpriseControls_DEV')



##The script willattempt to determine if the failure was connection related or sql related.
##First it will determine if the last run of the workflow was
## 1. Within the last hour
###########################################################
##set up command line arguments
## It is necessary to use 2 different types of quotations within the script
## because single ticks (') are used for the literals within the sql it is
## necessary to use (") outside of the whole script
#########################################################

def f_infa_repository_call(workflow):
#workflow=sys.argv[1]
    workflow="'"+workflow+"'"
    wf=workflow.replace("'","")

    print 'called  workflow   '+workflow


########################################
##Extract the worfklow date difference to determine that the job has run within the past hour
########################################
    cnxn=pyodbc.connect(insert)
    crsr=cnxn.cursor()
    sql_cmd="select datediff(hh,(select  max(actual_start) from REP_SESS_LOG  where workflow_name="+workflow+"),getdate())"
    rtrn=crsr.execute(sql_cmd)
    v_date_diff=crsr.fetchone()
    date_diff=v_date_diff[0]
    crsr.close()
######isnull(FIRST_ERROR_MSG,'no_first_error'),,FIRST_ERROR_CODE
#######################################################
##Determine the attributes of the last workflow run

    sql_cmd="select WORKFLOW_RUN_ID,successful_rows,subject_area,workflow_name,last_error_code, \
    isnull(last_error,'no_error'),isnull(FIRST_ERROR_MSG,'no_first_error'),isnull(first_error_code,0) as first_error_code from REP_SESS_LOG   \
    where workflow_name="+workflow+" and  actual_start in \
    (select  max(actual_start) from REP_SESS_LOG where workflow_name="+workflow+")"

##    sql_cmd="select WORKFLOW_RUN_ID,successful_rows,subject_area,workflow_name,isnull(last_error_code,0) as last_error_code, \
##    isnull(last_error,'None') as last_error  ,isnull(FIRST_ERROR_MSG,'None') as first_error , \
##    isnull(first_error_code,0) as first_error_code from REP_SESS_LOG   \
##    where workflow_name="+workflow+" and  actual_start in  \
##    (select  max(actual_start) from REP_SESS_LOG where workflow_name="+workflow+")"    
    cnxn=pyodbc.connect(insert)
    crsr2=cnxn.cursor()
    #crsr2=dbc.cursor()
    rtrn2=crsr2.execute(sql_cmd)
    v_status=crsr2.fetchone()
    rows_success=str(v_status[1])
    wf_run_id=str(v_status[0])
    last_error_code=str(v_status[4])
    last_error=v_status[5]
    first_err_message=v_status[6]
    first_err_code=v_status[7]
    #first_err_message='goodbye'
    

    crsr2.close()
    cnxn=pyodbc.connect(insert)
    crsr3=cnxn.cursor()
    #crsr3=dbc.cursor()
    sql_cmd="select convert(varchar,actual_start)as start from REP_SESS_LOG where WORKFLOW_RUN_ID ="+wf_run_id   #933807"

    rtrn3=crsr3.execute(sql_cmd)
    v_status_2=crsr3.fetchone()
    date_time=v_status_2


    crsr3.close()

    if int(rows_success)>0:
        email_output_1=str.ljust('The workflow name is',50)+  workflow+'\n'
        email_output_2=str.ljust('The number of hours since the last run ',50)+ str(date_diff)+' hours\n'
        email_output_3=str.ljust('The date \\time of the last run was     ',50)+str(date_time)+'\n'
        email_output_4=str.ljust('The number of successful rows          ',50)+str(rows_success)+'\n'
        email_output_5=str.ljust('The last_error code                        ',50)+str(last_error_code)+'\n'
        email_output_6=str.ljust('The last_error message                      ',50)+last_error+'\n'
        email_output_7=str.ljust('The first_error message                     ',50)+first_err_message+'\n'
        email_output_8=str.ljust('The first error code           '          ,50)+str(first_err_code)+'\n'
        email_output_9=str.ljust(str(rows_success),50)+'\n'
        email_output=email_output_1+email_output_2+email_output_3+email_output_4+email_output_5+email_output_6+email_output_7+email_output_8
        print email_output

##############################################
##Place holder for any sort of output
##Email output is shown as an example
##Communication to a service desk ticket or
##insertion into a recording database for example
#############################################
##        fromaddress='my_email.com'
##        toaddress='my_email.com'
##
##        msg="subject:Workflow error notification!\n\n"+email_output
##        ##The error implies that the application did not run and the failure was at the autosys level\n\n"+str(q)+"\n\n"+str(z)
##
##        server=smtplib.SMTP("My_server",25)
##
##        server.sendmail(fromaddress,toaddress,msg)


    return(rows_success)    
      




